In [4]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.neighbors import NearestNeighbors
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt
import mplcyberpunk
import warnings

plt.style.use('cyberpunk')
warnings.filterwarnings("ignore")
plt.rcParams['font.family'] = 'AppleGothic'
plt.figure(figsize=(15,7))

<Figure size 1080x504 with 0 Axes>

<Figure size 1080x504 with 0 Axes>

In [5]:
df=pd.read_csv("travel.csv")
df.head()

,id,title,addr,addr_area,addr_sigun,cat_large,cat_middle,cat_sub,cat1,cat2,grade,review_num,ratings
0,127480,가거도(소흑산도),전라남도 신안군 흑산면 가거도길 38-2,전라남도,신안군,자연,자연관광지,바다,자연관광지|바다,자연|자연관광지|바다,5,0,0.0
1,2660376,기점·소악도,전라남도 신안군 증도면,전라남도,신안군,자연,자연관광지,바다,자연관광지|바다,자연|자연관광지|바다,1,0,0.0
2,128703,깃대봉,전라남도 신안군 흑산면,전라남도,신안군,자연,자연관광지,산,자연관광지|산,자연|자연관광지|산,3,0,0.0
3,1560396,대둔도,전라남도 신안군 흑산면 김이수길 57,전라남도,신안군,자연,자연관광지,바다,자연관광지|바다,자연|자연관광지|바다,2,0,5.0
4,127766,도초 시목해수욕장,전라남도 신안군 도초면 시목길 219-30,전라남도,신안군,자연,자연관광지,바다,자연관광지|바다,자연|자연관광지|바다,4,1,4.0


In [6]:
df.isnull().sum()

id              0
title           0
addr            0
addr_area       0
addr_sigun      0
cat_large       0
cat_middle      0
cat_sub         0
cat1            0
cat2            0
grade           0
review_num     54
ratings       113
dtype: int64

In [10]:
# df=df.dropna(axis=0)

In [7]:
cv= CountVectorizer()
cat=cv.fit_transform(df.cat1)
cv.vocabulary_

{'자연관광지': 26,
 '바다': 13,
 '강계곡': 0,
 '사진맛집': 15,
 '체험관광지': 30,
 '이색': 25,
 '역사관광지': 21,
 '전통': 27,
 '건축': 2,
 '조형물': 28,
 '종교': 29,
 '휴양관광지': 34,
 '물놀이': 12,
 '공원': 3,
 '기념': 7,
 '예술': 22,
 '문화시설': 11,
 '육상레포츠': 23,
 '놀거리': 8,
 '수상레포츠': 18,
 '건강': 1,
 '농어촌': 9,
 '숙박시설': 19,
 '함께': 32,
 '쇼핑': 17,
 '시장': 20,
 '관광자원': 5,
 '산업관광지': 16,
 '레포츠소개': 10,
 '공장단지': 4,
 '복합레포츠': 14,
 '교육': 6,
 '음식점': 24,
 '항공레포츠': 33,
 '플렉스': 31}

In [8]:
cat1=pd.DataFrame(
        cat.toarray(),
        columns=list(sorted(cv.vocabulary_.keys(),key=lambda x : cv.vocabulary_[x]))
)
cat1

,강계곡,건강,건축,공원,공장단지,관광자원,교육,기념,놀거리,농어촌,...,이색,자연관광지,전통,조형물,종교,체험관광지,플렉스,함께,항공레포츠,휴양관광지
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19379,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19380,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19381,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19382,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
a= cat1.iloc[0,:]
b= cat1.iloc[1,:]
np.linalg.norm(a-b)

0.0

In [10]:

nbrs=NearestNeighbors(n_neighbors=10).fit(cat1)
nbrs


NearestNeighbors(n_neighbors=10)

In [11]:
df['addr_area'].loc[0]

'전라남도'

In [12]:
distances, indexes = nbrs.kneighbors([a])

recomandations =df.loc[indexes[0],["title"]]
recomandations["distance"]= distances[0]

recomandations

,title,distance
1447,삼양해수욕장,0.0
1464,수월봉과 차귀해안,0.0
1444,산지 등대,0.0
5774,대진항 (영덕),0.0
1438,비양도선착장,0.0
1457,세화해변,0.0
1471,알작지,0.0
1425,도두항,0.0
1473,엉알해안,0.0
1442,사수항,0.0


In [13]:
def sim_distance(data,n1,n2):
    sum=0
    for i in data.loc[n1,data.loc[n1,:]>=0].index:
        if data.loc[n2,i] >=0:
            sum+=pow(data.loc[n1,i]-data.loc[n2,i],2)
    return sqrt(1/(sum+1))

In [14]:
sim_distance(cat1,)

TypeError: sim_distance() missing 2 required positional arguments: 'n1' and 'n2'